# Imports

In [8]:
from arcgis.gis import GIS
import configparser
import os

# Variables

In [9]:
# Credentials access and variable creation
credentials_file = fr"..\Credentials\Credentials.cfg"
config_parser = configparser.ConfigParser()
config_parser.read(credentials_file)
md_hub_url = config_parser["DEFAULT"]["url_maps"]
md_hub_admin = config_parser["DEFAULT"]["login"]
md_hub_pwd = config_parser["DEFAULT"]["password"]

# Functions

In [10]:
def find_cloned_keywords(title):
    clone_keywords = ["backup", "copy", "copied", "clone", "cloned"]
    title = title.lower()
    for keyword in clone_keywords:
        if keyword in title:
            return True
    return False

In [11]:
def toggle_delete_protection(item_id, title):
    item_of_interest = gis.content.get(item_id)
    
    # Before
    delete_details_dict = item_of_interest.delete(force=False, dry_run=True)
    can_delete_item = delete_details_dict.get("can_delete")
    print(f"Delete:{can_delete_item}, Item: {title}")
    
    # Take action
    if not can_delete_item:
        item_of_interest.protect(enable=False)
    
    # After
    delete_details_dict = item_of_interest.delete(force=False, dry_run=True)
    can_delete_item = delete_details_dict.get("can_delete")
    print(f"\tDelete:{can_delete_item}")

    return        

In [ ]:
def delete_item(item_id, title):
    item_of_interest = gis.content.get(item_id)
    
    try:
        result = item_of_interest.delete()
    except RuntimeError as rte:
        result = rte

    print(f"Delete Item: {title}")
    print(f"\t{result}")

    return 

# Functionality

### Create a gis connection and get the users in the hub

In [12]:
gis = GIS(url=md_hub_url, username=md_hub_admin, password=md_hub_pwd)

### Get the mdimapdatacatalog user object

In [14]:
mdimapdatacatalog_user = gis.users.get("mdimapdatacatalog")

### Get the list of folders in the account

In [16]:
mdimap_folder_objs_list = mdimapdatacatalog_user.folders

### Get the content in the root folder

In [17]:
root_folder_content_list = mdimapdatacatalog_user.items(folder=None, max_items=5000)
len(root_folder_content_list)

1014

### Build a dictionary of items in the root folder with key: value of item id: item title

In [18]:
item_id_title_dict = {item_obj.id: item_obj.title for item_obj in root_folder_content_list}

### Search for clone related content using function

In [19]:
clone_related_content = ({item_id: title for item_id, title in item_id_title_dict.items() 
                          if find_cloned_keywords(title)})
len(clone_related_content)

0

In [20]:
clone_related_content

{}

### Toggle delete protection on the clone related content

In [21]:
for item_id, title in clone_related_content.items():
    toggle_delete_protection(item_id=item_id, title=title)

### Delete clone related content

In [22]:
for item_id, title in clone_related_content.items():
    delete_item(item_id=item_id, title=title)